# Wrappers example

In this notebook, we'll explore the wrappers defined by Sinergym and how to use them. You can also create your own wrappers by inheriting from *gym.Wrapper* or its variants.

In [1]:
import gymnasium as gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import *

## Multi-Objective Wrapper

[MO-Gymnasium](https://github.com/Farama-Foundation/MO-Gymnasium) is an open-source Python library for developing and comparing multi-objective reinforcement learning algorithms. These environments return a reward vector instead of a scalar value, one for each objective.

To be as general as possible, it could be beneficial for Sinergym to also provide that reward vector. This way, Sinergym would be compatible with both multi-objective algorithms and algorithms that work with a traditional reward value.

We can transform the returned reward into a vector using the following wrapper:

In [2]:
env=gym.make('Eplus-5zone-hot-discrete-v1')
env=MultiObjectiveReward(env,reward_terms=['energy_term','comfort_term'])

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function ini

Ensure that `reward_terms` are available in the `info` dict returned in the environment's step method. Otherwise, an execution error will occur.
By default, Sinergym environments return all reward terms specified in the used reward class in the `info` dict, so if the objective exists in the reward term, you shouldn't encounter any problems.

In [3]:
env.reset()
action = env.action_space.sample()
obs, reward, terminated, truncated, info = env.step(action)
env.close()
print(reward)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]
[-0.00589738497079933, -0.09533249490526607]


## Previous Observations Wrappers

This Wrapper will add the previous timestep's observation values to the current environment observation.
You can select the variables you want to track its previous observation values. 
The observation space will be updated with the new dimension.

In [4]:
env=gym.make('Eplus-5zone-hot-discrete-v1')
env = PreviousObservationWrapper(env, previous_variables=[
        'htg_setpoint',
        'clg_setpoint',
        'air_temperature'])

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function ini

You can see that the observation values have been updated:

In [5]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.get_wrapper_attr('observation_variables'),obs))
env.close()
print('NEW OBSERVATION: ',obs_dict)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]
NEW OBSERVATION:  {'month': np.float32(1.0), 'day_of_month': np.float32(1.0), 'hour': np.float32(0.0), 'outdoor_temperature': np.float32(4.8), 'outdoor_humidity': np.float32(61.0), 'wind_speed': np.float32(4.65), 'wind_direction': np.float32(160.0), 'diffuse_solar_radiation': np.float32(0.0), 'direct_solar_radiation': np.float32(0.0), 'htg_setpoint': np.float32(12.8), 'clg_setpoint': np.float32(40.0), 'air_temperature': np.float32(19.809336), 'air_humidity': np.float32(27.848707), 'people_occupant': np.float32(0.0), 'co2_emission': np.float32(0.0), 'HVAC_electricity_demand_rate': np.float32(117.9477), 'total_electricity_HVAC': np.float32(106152.93), 'htg_setpoint_previous': np.float32(12.8), 'clg_setpoint_previous': np.float32(40.0), 'air_temperature_previous': np.float32(19.95039)}


## Datetime Wrapper

This wrapper will replace the `day` value with the `is_weekend` flag, and `hour` and `month` with sin and cos values. 
The observation space is also automatically updated.

In [6]:
env=gym.make('Eplus-5zone-hot-discrete-v1')
env = DatetimeWrapper(env)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function ini

Specifically, this wrapper removes the observation variables `month`, `day`, and `hour`, and adds `month_sin`, `month_cos`, `is_weekend`, `hour_sin`, and `hour_cos` instead:

In [7]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.get_wrapper_attr('observation_variables'),obs))
env.close()
print('NEW OBSERVATION: ',obs_dict)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]
NEW OBSERVATION:  {'month_cos': np.float64(1.0), 'month_sin': np.float64(0.0), 'is_weekend': np.float64(0.0), 'hour_cos': np.float64(1.0), 'hour_sin': np.float64(0.0), 'outdoor_temperature': np.float64(4.800000190734863), 'outdoor_humidity': np.float64(61.0), 'wind_speed': np.float64(4.650000095367432), 'wind_direction': np.float64(160.0), 'diffuse_solar_radiation': np.float64(0.0), 'direct_solar_radiation': np.float64(0.0), 'htg_setpoint': np.float64(12.800000190734863), 'clg_setpoint': np.float64(40.0), 'air_temperature': np.float64(19.809335708618164), 'air_humidity': np.float64(27.84870719909668), 'people_occupant': np.float64(0.0), 'co2_emission': np.float64(0.0), 'HVAC_electricity_demand_rate': np.float64(117.94770050048828), 'total_electricity_HVAC': np.float64(106152.9296875)}


## Environment Action Normalization Wrapper

Here's an example of how to normalize a previous continuous environment action space. If we don't define the range values, it will default to the range `[-1,1]`:

In [8]:
# We will create a continuous environment
env=gym.make('Eplus-5zone-hot-continuous-v1')
print('ORIGINAL ACTION SPACE: ',env.get_wrapper_attr('action_space'))
# NORMALIZATION
# Apply the normalize action wrapper
env=NormalizeAction(env,normalize_range=(-1.0,1.0))
print('WRAPPED ACTION SPACE: ',env.get_wrapper_attr('action_space'))
env.reset()
for i in range(5):
    action=env.action_space.sample()
    print('Normalized action: ',action)
    _,_,_,_,info=env.step(action)
    print('Action done in simulator: ', info['action'])
env.close()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function

## Environment Discretization Wrapper

Here's an example of how to discretize a previous continuous environment. We'll need the **new discrete action space** and an **action mapping function** whose output matches the original unwrapped environment action space:

In [9]:
# We will create a continuous environment
env=gym.make('Eplus-5zone-hot-continuous-v1')
print('ORIGINAL ACTION SPACE: ',env.get_wrapper_attr('action_space'))
print('IS DISCRETE?: ',env.get_wrapper_attr('is_discrete'))
# DISCRETIZATION
# Defining new discrete space and action mapping function
new_discrete_space = gym.spaces.Discrete(10) # Action values [0,9]
def action_mapping_function(action):
    mapping = {
        0: [15, 30], # These lists matches with original action space
        1: [16, 29],
        2: [17, 28],
        3: [18, 27],
        4: [19, 26],
        5: [20, 25],
        6: [21, 24],
        7: [22, 23],
        8: [22, 22.5],
        9: [21, 22.5]
    }

    return mapping[action]
# Apply the discretize wrapper
env=DiscretizeEnv(env,discrete_space=new_discrete_space,action_mapping=action_mapping_function)
print('WRAPPED ACTION SPACE: ',env.get_wrapper_attr('action_space'))
print('IS DISCRETE?: ',env.get_wrapper_attr('is_discrete'))
env.reset()
for i in range(5):
    action=env.action_space.sample()
    print('ACTION DISCRETE: ',action)
    _,_,_,_,info=env.step(action)
    print('Action done in simulator: ', info['action'])
env.close()



#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-continuous-v1]


As seen in the output, the input is an int, but a list with the original action space is created in the simulator.

## Discrete Incremental Wrapper

This wrapper updates an environment for an incremental setpoint action space. It converts the environment into a *discrete* environment with an action mapping function and action space depending on the **step** and **delta** specified. The action is added to the **current setpoint** values instead of overwriting the latest action. Thus, the action is the current setpoint values with the increase, not the discrete value action that defines the increment/decrement itself.

In [10]:
env=gym.make('Eplus-5zone-hot-continuous-v1')
print('ORIGINAL ACTION SPACE: ',env.get_wrapper_attr('action_space'))
env = DiscreteIncrementalWrapper(
        env,initial_values=[21.0,25.0], delta_temp=2, step_temp=0.5)
print('WRAPPED ACTION SPACE: ',env.get_wrapper_attr('action_space'))
print('WRAPPED ACTION MAPPING: ',env.get_wrapper_attr('action_mapping'))

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function

The maximum and minimum values for creating the action mapping are read from the environment action space, ensuring that the setpoint increments and decrements do not exceed the agreed limits.
The delta and step values are used to determine how the discrete space of these increments and decrements will be constructed.
Here's an example of how it works:

In [11]:
env.reset()
print('CURRENT SETPOINTS VALUES: ', env.get_wrapper_attr('current_setpoints'))
for i in range(5):
    action=env.action_space.sample()
    _,_,_,_,info=env.step(action)
    print('Action number ',i,': ',env.get_wrapper_attr('action_mapping')(action))
    print('Setpoints update: ', info['action'])
env.close()

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-continuous-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-continu

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-continuous-v1]


## Normalization Wrapper

This wrapper is used to transform the observation received from the simulator into values between -1 and 1.
It's based on the [dynamic normalization wrapper of Gymnasium](https://gymnasium.farama.org/_modules/gymnasium/wrappers/normalize/#NormalizeObservation). Initially,
it may not be precise, and the values may often be out of range, so use this wrapper with caution.

In [12]:
#Original env
env=gym.make('Eplus-5zone-hot-discrete-v1')
env = NormalizeObservation(
        env=env)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function ini

In the following code, you can see how the specified variables have been correctly normalized:

In [13]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.get_wrapper_attr('observation_variables'),obs))
env.close()
print('OBSERVATION WITH NORMALIZATION: ',obs_dict)

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |***************************************************************************************************| 99%
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]
[WRAPPER NormalizeObservation] (INFO) : Saving normalization calibration data... [5zone-hot-discrete-v1]
OBSERVATION WITH NORMALIZATION:  {'month': np.float64(0.00499968750430059), 'day_of_month': np.float64(0.00499968750430059), 'hour': np.float64(0.0), 'outdoor_temperature': np.float64(0.9875907976185373), 'outdoor_humidity': np.float64(-0.9745626607331893), 'wind_speed': np.float64(0.9973969815420386), 'wind_direction': np.float64(0.9908525387095581), 'diffuse_solar_radiation': np.float64(0.0), 'direct_solar_radiation': np.float64(0.0), 'htg_setpoint': np.float64(0.007049581561774207), 'clg_setpoint': np.float64(0.0070688585864955075), 'air_temperature': np.float64(-0.44169349819860654), 'air_humidity': np.float64(0.1687716004048815), 'people_occupant': np.float64(0.0), 'co2_emission': np.float64(0.0), 

## Logging and storing data with logger wrappers

### LoggerWrapper layer

This wrapper uses Sinergym's logger storage class to capture the interaction flow with the environment (**LoggerStorage**), accumulating all the information. The class used by the wrapper can be replaced with a different back-end. It can then be combined with various wrappers to output the stored data, such as `CSVLogger` or `WandBLogger`. For more information about the *Sinergym* Logger, visit [Logging System Overview](https://ugr-sail.github.io/sinergym/compilation/main/pages/logging.html#logging-system-overview), [Logger Wrappers](https://ugr-sail.github.io/sinergym/compilation/main/pages/wrappers.html#logger-wrappers) and [an example about custom loggers](https://ugr-sail.github.io/sinergym/compilation/main/pages/notebooks/personalize_loggerwrapper.html).

In [3]:
env=gym.make('Eplus-5zone-hot-discrete-v1')
env=LoggerWrapper(env, storage_class=LoggerStorage)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function ini

This wrapper enables the use of a *LoggerStorage* instance within the environment class and automatically captures interaction data while actions are sent by an agent. At each reset, the data from this class is cleared to start the next episode. The idea is to combine it with other output loggers like those listed below:

### LoggerCSV layer

In [4]:
env=CSVLogger(env)

env.reset()    
truncated = terminated = False
current_month = 0
while not (terminated or truncated):
    a = env.action_space.sample()
    _,_,terminated,truncated,_=env.step(a)
env.close()

[WRAPPER CSVLogger] (INFO) : Wrapper initialized.
#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-discrete-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sin

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Progress: |***************************************************************************************************| 99%
[WRAPPER CSVLogger] (INFO) : Environment closed, data updated in monitor and progress.csv.
[ENVIRONMENT] (INFO) : Environment closed. [5zone-hot-discrete-v1]


Once the *LoggerWrapper* is applied, this wrapper can be used to output episode data through Sinergym’s output, along with summary metrics added to CSV files. More details on this structure can be found in [OutputFormat](https://ugr-sail.github.io/sinergym/compilation/main/pages/output.html). Sinergym will raise an error if this wrapper is used without first enabling *LoggerWrapper* or a custom logger.

### WandBLogger layer

In [2]:
# env=WandBLogger(env = Env,
#                 entity = <wandb_account_entity>,
#                 project_name = <wandb_project_name>,
#                 run_name = <run_name>
#                 group = 'Notebook_example',
#                 tags: ['tag1','tag2'],
#                 save_code = False,
#                 dump_frequency = 1000,
#                 artifact_save = True,
#                 artifact_type = 'output',
#                 excluded_info_keys = ['reward',
#                                   'action',
#                                   'timestep',
#                                   'month',
#                                   'day',
#                                   'hour',
#                                   'time_elapsed(hours)',
#                                   'reward_weight',
#                                   'is_raining'],
#                 excluded_episode_summary_keys = ['terminated',
#                                              'truncated']):

# env.reset()    
# truncated = terminated = False
# current_month = 0
# while not (terminated or truncated):
#     a = env.action_space.sample()
#     _,_,terminated,truncated,_=env.step(a)
# env.close()

Similar to *CSVLogger*, this wrapper requires the environment to have been previously encapsulated by a LoggerWrapper or any custom logger. The user must have a pre-existing Weights & Biases account and correctly specify the fields. This wrapper does not override *CSVLogger*; both can be applied simultaneously without issue.

## Multi Observation Wrapper

This wrapper stacks the observations received in a history queue (the size can be customized).

In [15]:
#Original environment
env=gym.make('Eplus-5zone-hot-discrete-v1')
obs, info=env.reset()
print('BEFORE MULTI OBSERVATION: ',obs)

#Multi Observation environment
env=MultiObsWrapper(env, n=5, flatten=True)
obs, info=env.reset()

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-discrete-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-discrete-v1-res2]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function ini

[SIMULATOR] (INFO) : handlers are ready.
[SIMULATOR] (INFO) : System is ready.
Progress: |***************************************************************************************************| 99%


In [16]:
print('AFTER MULTI OBSERVATION: ',obs)
env.close()

AFTER MULTI OBSERVATION:  [1.0000000e+00 1.0000000e+00 0.0000000e+00 5.1999998e+00 5.7000000e+01
 5.4250002e+00 1.7500000e+02 0.0000000e+00 0.0000000e+00 1.2800000e+01
 4.0000000e+01 1.9671045e+01 2.7883675e+01 0.0000000e+00 0.0000000e+00
 1.1794770e+02 1.0615293e+05 1.0000000e+00 1.0000000e+00 0.0000000e+00
 5.1999998e+00 5.7000000e+01 5.4250002e+00 1.7500000e+02 0.0000000e+00
 0.0000000e+00 1.2800000e+01 4.0000000e+01 1.9671045e+01 2.7883675e+01
 0.0000000e+00 0.0000000e+00 1.1794770e+02 1.0615293e+05 1.0000000e+00
 1.0000000e+00 0.0000000e+00 5.1999998e+00 5.7000000e+01 5.4250002e+00
 1.7500000e+02 0.0000000e+00 0.0000000e+00 1.2800000e+01 4.0000000e+01
 1.9671045e+01 2.7883675e+01 0.0000000e+00 0.0000000e+00 1.1794770e+02
 1.0615293e+05 1.0000000e+00 1.0000000e+00 0.0000000e+00 5.1999998e+00
 5.7000000e+01 5.4250002e+00 1.7500000e+02 0.0000000e+00 0.0000000e+00
 1.2800000e+01 4.0000000e+01 1.9671045e+01 2.7883675e+01 0.0000000e+00
 0.0000000e+00 1.1794770e+02 1.0615293e+05 1.000000

## Nesting wrappers

All wrappers available in Sinergym are stackable, organized in layers. However, the order in which these layers are applied can affect the final result, depending on the wrappers being used.

For instance, activating the logger before normalizing differs from doing it in the reverse order. In the first case, the data will be logged without normalization, even though the agent will operate in a normalized environment. In the second case, the logger will capture the normalized values since it encapsulates the normalization applied by the previous layer.

An example of how to nest wrappers is shown below:

In [17]:
env = gym.make('Eplus-5zone-hot-continuous-v1')
env = MultiObjectiveReward(
    env=env,
    reward_terms=[
        'energy_term',
        'comfort_term'])
env = PreviousObservationWrapper(env, previous_variables=[
    'htg_setpoint',
    'clg_setpoint',
    'air_temperature'])
env = DatetimeWrapper(env)
env = DiscreteIncrementalWrapper(
    env,initial_values=[21.0,25.0], delta_temp=2, step_temp=0.5)
env = NormalizeObservation(
    env=env)
env = LoggerWrapper(env=env)
env = MultiObsWrapper(env=env, n=5, flatten=True)

#==============================================================================================#
[ENVIRONMENT] (INFO) : Creating Gymnasium environment... [5zone-hot-continuous-v1]
#==============================================================================================#
[MODELING] (INFO) : Experiment working directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1]
[MODELING] (INFO) : Model Config is correct.
[MODELING] (INFO) : Updated building model with whole Output:Variable available names
[MODELING] (INFO) : Updated building model with whole Output:Meter available names
[MODELING] (INFO) : runperiod established: {'start_day': 1, 'start_month': 1, 'start_year': 1991, 'end_day': 31, 'end_month': 12, 'end_year': 1991, 'start_weekday': 0, 'n_steps_per_hour': 4}
[MODELING] (INFO) : Episode length (seconds): 31536000.0
[MODELING] (INFO) : timestep size (seconds): 900.0
[MODELING] (INFO) : timesteps per episode: 35041
[REWARD] (INFO) : Reward function

Now we simply use the environment with the wrappers, for example:

In [18]:
for i in range(1):
    obs, info = env.reset()
    truncated = terminated = False
    current_month = 0
    while not (terminated or truncated):
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', reward, info)
env.close()

#----------------------------------------------------------------------------------------------#
[ENVIRONMENT] (INFO) : Starting a new episode... [5zone-hot-continuous-v1] [Episode 1]
#----------------------------------------------------------------------------------------------#
[MODELING] (INFO) : Episode directory created [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1/Eplus-env-sub_run1]
[MODELING] (INFO) : Weather file USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw used.
[MODELING] (INFO) : Adapting weather to building model. [USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw]
[ENVIRONMENT] (INFO) : Saving episode output path... [/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1/Eplus-env-sub_run1/output]
[SIMULATOR] (INFO) : Running EnergyPlus with args: ['-w', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-continuous-v1-res1/Eplus-env-sub_run1/USA_AZ_Davis-Monthan.AFB.722745_TMY3.epw', '-d', '/workspaces/sinergym/examples/Eplus-env-5zone-hot-continu

/usr/local/lib/python3.12/dist-packages/gymnasium/spaces/box.py:240: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Reward:  [-0.260786748513617, 0.0] {'time_elapsed(hours)': 744.25, 'month': 2, 'day': 1, 'hour': 0, 'is_raining': False, 'action': [np.float64(22.25), np.float64(27.5)], 'timestep': 2977, 'reward': -0.260786748513617, 'energy_term': -0.260786748513617, 'comfort_term': 0.0, 'reward_weight': 0.5, 'abs_energy_penalty': -5215.7349702723395, 'abs_comfort_penalty': 0, 'total_power_demand': 5215.7349702723395, 'total_temperature_violation': 0.0}
Reward:  [-0.00589738497079933, 0.0] {'time_elapsed(hours)': 1416.25, 'month': 3, 'day': 1, 'hour': 0, 'is_raining': False, 'action': [np.float64(20.5), np.float64(27.25)], 'timestep': 5665, 'reward': -0.00589738497079933, 'energy_term': -0.00589738497079933, 'comfort_term': 0.0, 'reward_weight': 0.5, 'abs_energy_penalty': -117.9476994159866, 'abs_comfort_penalty': 0, 'total_power_demand': 117.9476994159866, 'total_temperature_violation': 0.0}
Reward:  [-0.00589738497079933, 0.0] {'time_elapsed(hours)': 2160.25, 'month': 4, 'day': 1, 'hour': 0, 'is_ra